In [1]:
import pandas as pd
import numpy as np

In [2]:
dev_nominal = pd.read_csv("dev_data_binarylist.csv", encoding = 'utf-8')

In [3]:
dev_nominal.shape

(1836, 3118)

In [4]:
columns = dev_nominal.columns
columns_other = dev_nominal.columns[0:18]
columns = [x for x in columns if "-" in x]
aggregate_events = [1,11,21,31,41,51,61,71,81,91]
aggregate_dic = {}
for a in aggregate_events:
    raw_list = []
    for i in range(a,a+10):
        col = [x.split("-")[1] for x in columns if str(i)+"-" in x]
        raw_list.extend(col)
    raw_list= list(set(raw_list))
    aggregate_dic[a] = raw_list

In [5]:
aggregate_dic

{1: ['HandBall_EventType_binary',
  'FreeKick_Situation_binary',
  'FreeKickWon_EventType_binary',
  'Pass_AssistMethod_binary',
  'HeadedPass_AssistMethod_binary',
  'OffTarget_ShotOutcome_binary',
  'home_event',
  'SecondYellowCard_EventType_binary',
  'PenaltyConceded_EventType_binary',
  'OpenPlay_Situation_binary',
  'Offside_EventType_binary',
  'HitThePost_ShotOutcome_binary',
  'Foul_EventType_binary',
  'StraightRedCard_EventType_binary',
  'Corner_Situation_binary',
  'KeyPass_EventType2_binary',
  'ThroughBall_AssistMethod_binary',
  'OwnGoal_EventType2_binary',
  'YellowCard_EventType_binary',
  'time',
  'Blocked_ShotOutcome_binary',
  'Substitution_EventType_binary',
  'Corner_EventType_binary',
  'SendingOff_EventType2_binary',
  'None_AssistMethod_binary',
  'fast_break',
  'SetPiece_Situation_binary',
  'Cross_AssistMethod_binary',
  'FailedThroughBall_EventType2_binary',
  'OnTarget_ShotOutcome_binary',
  'Attempt(shot)_EventType_binary'],
 11: ['HandBall_EventType_b

In [6]:
def process_row (row,aggregate_dic):
    row_dict = {}
    for k in aggregate_dic.keys():
        columns = aggregate_dic[k]
        for i in range(k,k+10):
            for c in columns:
                c_name = str(k)+"_"+str(k+10)+"_"+c.replace("_binary","")
                if row[str(i)+"-"+c] > 0: 
                    if c_name in row_dict.keys():
                        row_dict[c_name] = row_dict[c_name] + int(row[str(i)+"-"+c])
                    else:
                        row_dict[c_name] =int(row[str(i)+"-"+c])
    return row_dict

In [7]:
row_dict = process_row(dev_nominal.iloc[[0]],aggregate_dic)
row_dict

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [16]:
dev_nominal.iloc[[0]]

,id_odsp,season,fthg,ftag,odd_h,odd_d,odd_a,class,D1_league_binary,E0_league_binary,...,100-OnTarget_ShotOutcome_binary,100-Cross_AssistMethod_binary,100-HeadedPass_AssistMethod_binary,100-None_AssistMethod_binary,100-Pass_AssistMethod_binary,100-ThroughBall_AssistMethod_binary,100-Corner_Situation_binary,100-FreeKick_Situation_binary,100-OpenPlay_Situation_binary,100-SetPiece_Situation_binary
0,GIqnQgzL/,2016,1,1,1.58,4.6,7.7,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
df_new = pd.DataFrame()
for index, row in dev_nominal.iterrows():
    row_dict = process_row(row,aggregate_dic)
    new_row = pd.DataFrame(row_dict, index=[index,])
    for c in columns_other:
        new_row[c] = row[c]
    df_new =  pd.concat([df_new, new_row])

In [48]:
df_new= df_new.sort_index(axis = 1)

In [49]:
df_new.to_csv("dev_data_aggregate.csv", encoding = 'utf-8',index=False)